## <span style='color:#ff5f27'> 📝 Imports

In [1]:
!pip install -r requirements.txt --quiet

In [2]:
from functions.llm_chain import load_model, get_llm_chain, generate_response

## <span style="color:#ff5f27;"> 🔮 Connect to Hopsworks Feature Store </span>

In [3]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://snurran.hops.works/p/11394
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27;"> ⚙️ Feature View Retrieval</span>

In [4]:
# Retrieve the 'bitcoin_feature_view' feature view
feature_view = fs.get_feature_view(
    name='bitcoin_feature_view',
    version=1,
)

# Initialize batch scoring
feature_view.init_batch_scoring(1)

## <span style="color:#ff5f27;">🪝 Retrieve Bitcoin Price Predictor Deployment</span>

In [5]:
# Retrieve model serving
ms = project.get_model_serving()

# Retrieve bitcoin predictor
model_deployment = ms.get_deployment("btcmodeldeployment")

Connected. Call `.close()` to terminate connection gracefully.


## <span style='color:#ff5f27'>⬇️ LLM Loading

In [6]:
# Load the LLM and its corresponding tokenizer.
model_llm, tokenizer = load_model()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


2024-04-20 23:37:50,528 INFO: We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
!nvidia-smi

Sat Apr 20 23:38:31 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:2D:00.0 Off |                  Off |
| 34%   63C    P2    82W / 300W |  29496MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## <span style='color:#ff5f27'>⛓️ LangChain

In [8]:
# Create and configure a language model chain.
llm_chain = get_llm_chain(
    model_llm,
    tokenizer,
)

## <span style='color:#ff5f27'>🧬 Model Inference


In [9]:
QUESTION = "Hi!"

response = generate_response(
    QUESTION,
    feature_view,
    model_deployment,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response)

🗓️ Today's date: Saturday, 2024-04-20
📖 

Hello! Today, Bitcoin closed at $25,000. The opening price was $24,500, and the volume was $1.5 billion. The highest price during the day was $25,500, while the lowest price was $24,000. The predicted price for tomorrow is $26,000.

The Bitcoin market has been showing signs of stability lately, with a relatively low volatility. However, it's always important to keep an eye on the market and be prepared for sudden changes.


In [10]:
QUESTION = "Who are you?"

response = generate_response(
    QUESTION,
    feature_view,
    model_deployment,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response)

🗓️ Today's date: Saturday, 2024-04-20
📖 

I am a highly skilled Bitcoin market analyst with extensive knowledge of the cryptocurrency industry. I specialize in providing insights and analysis on Bitcoin's performance and trends.


In [ ]:
import datetime

date_start = "2024-01-10"
date_end = "2024-01-11"

# Convert date strings to datetime objects
date_start_dt = datetime.datetime.strptime(date_start, "%Y-%m-%d").date()
date_end_dt = datetime.datetime.strptime(date_end, "%Y-%m-%d").date()

# Retrieve batch data for the specified date range
print(f"-- Getting batch data...")
batch_data = feature_view.get_batch_data(
    start_time=date_start_dt,
    end_time=date_end_dt# + datetime.timedelta(days=1),
)

In [ ]:
dt_unix = datetime.datetime.fromtimestamp(batch_data["unix"][0] / 1000, datetime.timezone.utc)

def convert_unix_to_date(x):
    x //= 1000
    x = datetime.datetime.fromtimestamp(x)
    return datetime.datetime.strftime(x, "%Y-%m-%d")

convert_unix_to_date(batch_data["unix"][0])

In [ ]:
batch_data["unix"]

In [ ]:
batch_data["date"] = batch_data["unix"].apply(convert_unix_to_date)
batch_data.drop(columns=["unix"], inplace=True)

In [ ]:
batch_data["date"]

In [ ]:
QUESTION = "What was the bitcoin opening and closing prices the 2024-01-10?"

response = generate_response(
    QUESTION, 
    feature_view,
    model_deployment,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response)

In [ ]:
QUESTION1 = "What was the bitcoin price trend from 2024-01-10 till 2024-01-14?"

response = generate_response(
    QUESTION, 
    feature_view,
    model_deployment,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response)

In [ ]:
QUESTION = "When and what was the maximum bitcoin price from 2024-03-10 till today?"

response = generate_response(
    QUESTION, 
    feature_view, 
    model_deployment,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response)

In [ ]:
QUESTION = "When and what was the maximum bitcoin price this month?"

response = generate_response(
    QUESTION, 
    feature_view, 
    model_deployment,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response)

In [ ]:
QUESTION = "What was the Bitcoin volume yesterday?"

response = generate_response(
    QUESTION,
    feature_view,
    model_deployment,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response)

In [ ]:
QUESTION = "What was the Bitcoin closing price yesterday?"

response = generate_response(
    QUESTION,
    feature_view,
    model_deployment,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response)

In [11]:
QUESTION = "What will the Bitcoin predicted price be tomorrow?"

response = generate_response(
    QUESTION,
    feature_view,
    model_deployment,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response)

🗓️ Today's date: Saturday, 2024-04-20
📖 Bitcoin price prediction on the 2024-04-21:
Date: 2024-04-20; Predicted price: 63520.159626345965

Based on our analysis, the predicted Bitcoin price for tomorrow, 2024-04-21, is expected to be around $63,520.16. However, it's essential to remember that the cryptocurrency market is highly volatile, and prices can fluctuate significantly. Always exercise caution when investing in Bitcoin or any other cryptocurrency.


---

## <span style='color:#ff5f27'>🧬 Inference with OpenAI


In [ ]:
from openai import OpenAI
import os
import getpass

from functions.llm_chain import generate_response_openai

In [ ]:
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass.getpass('🔑 Enter your OpenAI API key: ')

client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
)

In [ ]:
QUESTION = "What was the air quality like last week in London?"

response = generate_response_openai(   
    QUESTION,
    feature_view,
    model_deployment,
    client,
    verbose=True,
)
print(response)

In [ ]:
QUESTION4 = "What will the air quality be like in Chicago tomorrow?"

response4 = generate_response_openai(
    QUESTION4,
    feature_view,
    model_deployment,
    client,
    verbose=True,
)

print(response4)

---